In [26]:
import numpy as np
import matplotlib.pyplot as plt
from pingouin import pairwise_ttests
from pingouin import anova
import pingouin as pg
import pandas as pd
import scipy.stats as t


In [27]:
data = pd.read_csv(r"C:\Users\DELL2025\Downloads\Compressed\healthcare_drug_efficacy.csv")
data.head()

,Patient_ID,Drug_A_Time,Drug_B_Time,Dosage_50mg,Dosage_100mg,Dosage_150mg
0,1,5,7,6,5,4
1,2,6,8,7,6,5
2,3,7,6,8,7,6
3,4,5,9,6,5,4
4,5,6,7,7,6,5


In [28]:
data.isna().sum()

Patient_ID      0
Drug_A_Time     0
Drug_B_Time     0
Dosage_50mg     0
Dosage_100mg    0
Dosage_150mg    0
dtype: int64

In [29]:
data.describe()

,Patient_ID,Drug_A_Time,Drug_B_Time,Dosage_50mg,Dosage_100mg,Dosage_150mg
count,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.50000,6.250000,7.550000,6.500000,6.250000,5.250000
std,5.91608,1.069924,1.099043,0.945905,1.069924,1.069924
min,1.00000,5.000000,6.000000,5.000000,5.000000,4.000000
25%,5.75000,5.000000,7.000000,6.000000,5.000000,4.000000
50%,10.50000,6.000000,7.500000,6.500000,6.000000,5.000000
75%,15.25000,7.000000,8.250000,7.000000,7.000000,6.000000
max,20.00000,8.000000,9.000000,8.000000,8.000000,7.000000


In [30]:
#t-test
t_data = data[['Drug_A_Time', 'Drug_B_Time']]
t_data 

,Drug_A_Time,Drug_B_Time
0,5,7
1,6,8
2,7,6
3,5,9
4,6,7
5,8,8
6,7,7
7,6,9
8,5,6
9,7,8


In [31]:
mean_drug_a = t_data['Drug_A_Time'].mean()
mean_drug_b = t_data['Drug_B_Time'].mean()
print(mean_drug_a, mean_drug_b)


6.25 7.55


In [32]:
std_drug_a = t_data['Drug_A_Time'].std()
std_drug_b = t_data['Drug_B_Time'].std()
print(std_drug_a, std_drug_b)

1.0699237552766379 1.0990426455975697


In [33]:
t_val= (mean_drug_a - mean_drug_b) / np.sqrt((std_drug_a**2/len(t_data['Drug_A_Time'])) + (std_drug_b**2/len(t_data['Drug_B_Time'])))
t_val

np.float64(-3.790368106175599)

In [34]:
df = len(t_data['Drug_A_Time']) + len(t_data['Drug_B_Time']) - 2
crit_t = 2.025
if t_val > crit_t:
    print("Reject the null hypothesis")
else:    
    print("Fail to reject the null hypothesis") 

Fail to reject the null hypothesis


In [35]:
p_value = 1 - t.t.cdf(t_val, df)
p_value

np.float64(0.9997384075091365)

In [36]:
alpha  = 0.05 

if alpha > p_value:
    print("Reject the null hypothesis")
else:    
    print("Fail to reject the null hypothesis") 

Fail to reject the null hypothesis


In [37]:
#ANOVA
data_nova = data[['Dosage_50mg', 'Dosage_100mg', 'Dosage_150mg']]
data_nova

,Dosage_50mg,Dosage_100mg,Dosage_150mg
0,6,5,4
1,7,6,5
2,8,7,6
3,6,5,4
4,7,6,5
5,5,8,7
6,6,7,6
7,7,6,5
8,8,5,4
9,6,7,6


In [38]:
mean_dose_50 = data_nova['Dosage_50mg'].mean()
mean_dose_100 = data_nova['Dosage_100mg'].mean()
mean_dose_150 = data_nova['Dosage_150mg'].mean()
sum_dose_50 = data_nova['Dosage_50mg'].sum()
sum_dose_100 = data_nova['Dosage_100mg'].sum()
sum_dose_150 = data_nova['Dosage_150mg'].sum()

print(mean_dose_50, mean_dose_100, mean_dose_150)
print(sum_dose_50, sum_dose_100, sum_dose_150)

6.5 6.25 5.25
130 125 105


In [39]:
data_n = data_nova.melt(var_name='Dosage', value_name='Time_days')
data_n.head()

,Dosage,Time_days
0,Dosage_50mg,6
1,Dosage_50mg,7
2,Dosage_50mg,8
3,Dosage_50mg,6
4,Dosage_50mg,7


In [40]:
sum_data = data_n['Time_days'].sum()
mean_data = data_n['Time_days'].mean()

print(sum_data, mean_data)

360 6.0


In [41]:
ssbtw = sum_data * (mean_dose_50 - mean_data)**2 + sum_data * (mean_dose_100 - mean_data)**2 + sum_data * (mean_dose_150 - mean_data)**2
sswthin = ((data['Dosage_50mg']- mean_dose_50 )**2).sum() + ((data['Dosage_100mg']- mean_dose_100 )**2).sum() + ((data['Dosage_150mg']- mean_dose_150 )**2).sum()
msbtw = ssbtw / 2
mswthin = sswthin / 57
f_val = msbtw / mswthin
print(f_val)

148.38842975206612


In [44]:
dif_btw_gps = pg.anova(data=data_n, dv='Time_days', between='Dosage')
print(dif_btw_gps)

   Source  ddof1  ddof2         F     p-unc       np2
0  Dosage      2     57  8.243802  0.000717  0.224359


In [45]:
alpha = 0.05
if 0.000717 < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

Reject the null hypothesis


In [46]:
dbg = pg.pairwise_tests(data=data_n, dv='Time_days', between='Dosage', padjust='bonferroni')
dbg

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Dosage,Dosage_100mg,Dosage_150mg,False,True,2.955610,38.0,two-sided,0.005336,0.016007,bonferroni,8.091,0.916077
1,Dosage,Dosage_100mg,Dosage_50mg,False,True,-0.782881,38.0,two-sided,0.438549,1.000000,bonferroni,0.394,-0.242650
2,Dosage,Dosage_150mg,Dosage_50mg,False,True,-3.914407,38.0,two-sided,0.000364,0.001091,bonferroni,72.268,-1.213251


The 'p-unc' values represent the uncorrected p-values for the pairwise comparisons of dosage pairs.
After applying the Bonferroni correction ('p-corr'), the adjusted p-values account for multiple comparisons,
reducing the likelihood of Type I errors. This helps to identify significant differences between dosage pairs
while controlling the family-wise error rate.

2 pairs have p-unc less than alpha which indicate that they are statsistcally significant 